# Project 1: Machine Learning

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import json
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from typing import Tuple
from helper import load_csv_data
from processing import *
from implementations import *
from feature_expansion import *
from crossvalidation import *
from metrics import f1_score, mse_loss

# Import of the data
Import of the train and test data

In [2]:
y_train, x_train, id_train = load_csv_data("../data/train.csv")
y_test , x_test , id_test  = load_csv_data("../data/test.csv")
# print('train data shape: ', x_train.shape, y_train.shape)
# print('test  data shape: ', x_test.shape, y_test.shape)
with open('../col_name.json', 'r') as file:
    features = json.load(file)['col_names']

# Preprocess data
We pre process the data to get a clean dataset

In [3]:
x_train_cleaned = standardize(clean_data(x_train, features))

We then divide the dataset depending on the Pri_Jet_number feature which can take values 0, 1, 2 or 3. Since the number values that are equal to 3 is really small, we will combine it with the values which have 2 so we will have a 3 subsets

### Feature expansion
We will now do feature engineering to increase the results we will have. We do degree root transformation, polynomial transformation, logarithmic transformation and reciprocical transformation.

In [ ]:
x_train_finished = build_new_x(x_train_cleaned)

# Cross-Validation Pipeline

cross validation pipeline. we will use 5-fold cross validation for choosing the optimal parameters.
below is the list of models we will use throughout this process.

1. least_squares (no parameter tuning needed.)
2. ridge LS 
3. mse_gd
4. mse_sgd
5. logistic
6. reg_logistic

we will optimise the weigth based on the mse loss. But the selection process will be based on observing F1 score on validation set.

To see the effect of data manipulation, we will try 3 sets of data.
1. cleaned data
2. standardized data
3. feature-engineered data

From this process, we would expect the 3rd trial would give us the best result.

In [145]:
from itertools import product

class HyperParameterTuner:
    def __init__(
        self, 
        x: np.ndarray, 
        y: np.ndarray, 
        model_name: str,
        num_folds: int=5,
        num_seed: int=0,
        max_iter: int=10
    ):

        available_models = {
            'least_squares': least_squares,
            'ridge': ridge_regression,
            'mse_gd': mean_squared_error_gd,
            'mse_sgd': mean_squared_error_sgd,
            'logistic': logistic_regression,
            'reg_logistic': reg_logistic_regression
        }

        self.x = x
        self.y = y
        self.model = available_models[model_name]
        self.model_name = model_name
        self.num_folds = num_folds
        self.num_seed  = num_seed
        self.max_iter  = max_iter
        
        # build k_indices
        np.random.seed(self.num_seed)
        self.build_k_indices()

        # get model params given model specs.
        model_parameters = {
            'least_squares': {}, 
            'ridge'        : {'lambda_': None},
            'mse_gd'       : {'initial_w': np.ones((x.shape[1],)), 'max_iters': self.max_iter, 'gamma': None},
            'mse_sgd'      : {'initial_w': np.ones((x.shape[1],)), 'max_iters': self.max_iter, 'gamma': None},
            'logistic'     : {'initial_w': np.ones((x.shape[1],)), 'max_iters': self.max_iter, 'gamma': None},
            'reg_logistic' : {'initial_w': np.ones((x.shape[1],)), 'max_iters': self.max_iter, 'gamma': None, 'lambda_': None},
        }
        self.hyp_params = model_parameters[model_name]

    def tune_(self) -> Tuple[list, float]:
        """
        hyperparameter tuning done by grid search.
        best parameters are found by finding the maximum f1 scores.
        """

        lambdas = np.logspace(-15,    2, 50)
        gammas  = np.logspace(-6 ,-0.25, 50)     

        f1_scores = np.array([])
        params = self.hyp_params
        # cross validation
        if self.model_name == 'least_squares':
            results = np.array([[k] + self.cross_validation_per_k(k, self.hyp_params)[-1] for k in range(self.num_folds)])
            return results[np.argmax(results[:,-1])]   
        
        elif self.model_name == 'reg_logistic':
            lambda_and_gammas = product(gammas, lambdas)
            for i, (gamma, lambda_) in enumerate(lambda_and_gammas):
                params['gamma'], params['lambda_'] = gamma, lambda_
                results = np.array([self.cross_validation_per_k(k, params) for k in range(self.num_folds)])
                f1_scores = np.append(f1_scores, np.mean(results, axis=0)[-1])
                
                print(f'Progress {i}/{50*50}, lambda_: {lambda_},  gamma: {gamma}, f1: {np.round(np.mean(results, axis=0)[-1], 4)}')            
                       
            f1_scores[np.isnan(f1_scores)] = 0 
            optimum_idx = np.argmax(f1_scores)
            best_params, best_f1 = lambda_and_gammas[optimum_idx], f1_scores[optimum_idx]
            return {'params' :best_params, 'f1_score' :best_f1}     

        elif self.model_name == 'ridge':
            for i, lambda_ in enumerate(lambdas):
                params['lambda_'] =lambda_
                results = np.array([self.cross_validation_per_k(k, params) for k in range(self.num_folds)])
                f1_scores = np.append(f1_scores, np.mean(results, axis=0)[-1])
                print(f'Progress {i}/{len(lambdas)}, lambda_: {lambda_}, f1: {np.round(np.mean(results, axis=0)[-1], 4)}')            
            
            f1_scores[np.isnan(f1_scores)] = 0 
            optimum_idx = np.argmax(f1_scores)
            best_params, best_f1 = lambdas[optimum_idx], f1_scores[optimum_idx]
            return {'lambda_' :best_params, 'f1_score' :best_f1}     
        else:
            for i, gamma in enumerate(gammas):
                
                params['gamma'] = gamma
                results = np.array([self.cross_validation_per_k(k, params) for k in range(self.num_folds)])
                f1_scores = np.append(f1_scores, np.mean(results, axis=0)[-1])
                print(f'Progress {i}/{len(gammas)}, gamma :{gamma} f1: {np.round(np.mean(results, axis=0)[-1], 4)}')
            # set nan values to 0.
            f1_scores[np.isnan(f1_scores)] = 0 
            
            # get optimum index
            optimum_idx = np.argmax(f1_scores)
            best_params, best_f1 = gammas[optimum_idx], f1_scores[optimum_idx]
            return {'gamma' :best_params, 'f1_score' :best_f1}
        
    def cross_validation_per_k(self, k: int, params: dict) -> list:
        """return the loss of given model."""
        # get k'th subgroup in test, others in train
        tr_indices, te_indices = self.k_indices[~(np.arange(self.k_indices.shape[0]) == k)].reshape(-1),\
                                 self.k_indices[k]
        
        # split the data based on train and validation indices
        y_trn, y_val = self.y[tr_indices], self.y[te_indices]
        x_trn, x_val = self.x[tr_indices], self.x[te_indices]

        # run the model
        params['tx'], params['y'] = x_trn, y_trn
        w, _ = self.model(**params)
        
        # calculate the loss for train and test data
        loss_trn = np.sqrt(mse_loss(y_trn, x_trn, w))
        loss_val = np.sqrt(mse_loss(y_val, x_val, w))
        
        # get validation f1-score
        y_pred = get_classification_pred(x_val, w)
        f1_val = f1_score(y_val, y_pred)
        return [loss_trn, loss_val, f1_val]

    def build_k_indices(self):
        """
        build k indices for k-fold.
        Args:
            y:      shape=(N,)
            k_fold: K in K-fold, i.e. the fold num
            seed:   the random seed

        Returns:
            A 2D array of shape=(k_fold, N/k_fold) that indicates the data indices for each fold
        """
        num_row  = self.y.shape[0]
        interval = int(num_row / self.num_folds)
        indices  = np.random.permutation(num_row)

        self.k_indices = np.array([indices[k * interval : (k + 1) * interval] for k in range(self.num_folds)])
   


In [133]:
tuner = HyperParameterTuner(x_train_cleaned, y_train, 'mse_gd', max_iter=10)
tuner.tune_()

Progress 0/50, gamma :1e-06 f1: 0.44
Progress 1/50, gamma :1.3102281887548672e-06 f1: 0.44
Progress 2/50, gamma :1.7166979066078603e-06 f1: 0.44
Progress 3/50, gamma :2.2492659888140894e-06 f1: 0.44
Progress 4/50, gamma :2.94705170255181e-06 f1: 0.44
Progress 5/50, gamma :3.861310214401406e-06 f1: 0.44
Progress 6/50, gamma :5.059197488435822e-06 f1: 0.44
Progress 7/50, gamma :6.628703161826441e-06 f1: 0.44
Progress 8/50, gamma :8.68511373751352e-06 f1: 0.44
Progress 9/50, gamma :1.1379480841432356e-05 f1: 0.44
Progress 10/50, gamma :1.490971657184063e-05 f1: 0.44
Progress 11/50, gamma :1.9535130938771177e-05 f1: 0.44
Progress 12/50, gamma :2.559547922699533e-05 f1: 0.44
Progress 13/50, gamma :3.353591838789893e-05 f1: 0.44
Progress 14/50, gamma :4.393970560760795e-05 f1: 0.44
Progress 15/50, gamma :5.757104089267825e-05 f1: 0.44
Progress 16/50, gamma :7.543120063354622e-05 f1: 0.4399
Progress 17/50, gamma :9.883208538169628e-05 f1: 0.4399
Progress 18/50, gamma :0.0001294925842205263 f1

/Users/mac/Desktop/PycharmProjects/MLprojec1/scripts/implementations.py:133: RuntimeWarning: overflow encountered in exp
  return np.exp(t) / (1 + np.exp(t))
/Users/mac/Desktop/PycharmProjects/MLprojec1/scripts/implementations.py:133: RuntimeWarning: invalid value encountered in divide
  return np.exp(t) / (1 + np.exp(t))


Progress 48/50, gamma :0.42919342601287785 f1: 0.2401
Progress 49/50, gamma :0.5623413251903491 f1: 0.0075


{'gamma': 0.25001103826179305, 'f1_score': 0.562109789330598}

In [142]:
tuner = HyperParameterTuner(x_train_cleaned, y_train, 'mse_sgd', max_iter=10)
tuner.tune_()

ValueError: operands could not be broadcast together with shapes (200000,) (200000,29) 

In [146]:
tuner = HyperParameterTuner(x_train_cleaned, y_train, 'ridge', max_iter=10)
tuner.tune_()

Progress 0/50, lambda_: 1e-15, f1: 0.6648
Progress 1/50, lambda_: 2.222996482526191e-15, f1: 0.6647
Progress 2/50, lambda_: 4.9417133613238385e-15, f1: 0.6647
Progress 3/50, lambda_: 1.0985411419875573e-14, f1: 0.6648
Progress 4/50, lambda_: 2.4420530945486547e-14, f1: 0.6647
Progress 5/50, lambda_: 5.4286754393238596e-14, f1: 0.6647
Progress 6/50, lambda_: 1.2067926406393265e-13, f1: 0.6647
Progress 7/50, lambda_: 2.682695795279727e-13, f1: 0.6647
Progress 8/50, lambda_: 5.963623316594636e-13, f1: 0.6647
Progress 9/50, lambda_: 1.3257113655901109e-12, f1: 0.6647
Progress 10/50, lambda_: 2.9470517025518096e-12, f1: 0.6647
Progress 11/50, lambda_: 6.551285568595496e-12, f1: 0.6647
Progress 12/50, lambda_: 1.4563484775012445e-11, f1: 0.6647
Progress 13/50, lambda_: 3.237457542817653e-11, f1: 0.6647
Progress 14/50, lambda_: 7.196856730011528e-11, f1: 0.6647
Progress 15/50, lambda_: 1.5998587196060573e-10, f1: 0.6647
Progress 16/50, lambda_: 3.5564803062231214e-10, f1: 0.6647
Progress 17/5

{'lambda_': 1e-15, 'f1_score': 0.6647511072771948}

In [147]:
tuner = HyperParameterTuner(x_train_cleaned, y_train, 'logistic', max_iter=10)
tuner.tune_()

Progress 0/50, gamma :1e-06 f1: 0.44
Progress 1/50, gamma :1.3102281887548672e-06 f1: 0.44
Progress 2/50, gamma :1.7166979066078603e-06 f1: 0.44
Progress 3/50, gamma :2.2492659888140894e-06 f1: 0.44
Progress 4/50, gamma :2.94705170255181e-06 f1: 0.44
Progress 5/50, gamma :3.861310214401406e-06 f1: 0.44
Progress 6/50, gamma :5.059197488435822e-06 f1: 0.44
Progress 7/50, gamma :6.628703161826441e-06 f1: 0.44
Progress 8/50, gamma :8.68511373751352e-06 f1: 0.44
Progress 9/50, gamma :1.1379480841432356e-05 f1: 0.44
Progress 10/50, gamma :1.490971657184063e-05 f1: 0.44
Progress 11/50, gamma :1.9535130938771177e-05 f1: 0.44
Progress 12/50, gamma :2.559547922699533e-05 f1: 0.44
Progress 13/50, gamma :3.353591838789893e-05 f1: 0.44
Progress 14/50, gamma :4.393970560760795e-05 f1: 0.44
Progress 15/50, gamma :5.757104089267825e-05 f1: 0.44
Progress 16/50, gamma :7.543120063354622e-05 f1: 0.44
Progress 17/50, gamma :9.883208538169628e-05 f1: 0.44
Progress 18/50, gamma :0.0001294925842205263 f1: 0.

{'gamma': 0.5623413251903491, 'f1_score': 0.5524871711629684}

In [148]:
tuner = HyperParameterTuner(x_train_cleaned, y_train, 'reg_logistic', max_iter=10)
tuner.tune_()

Progress 0/2500, lambda_: 1e-15,  gamma: 1e-06, f1: 0.44
Progress 1/2500, lambda_: 2.222996482526191e-15,  gamma: 1e-06, f1: 0.44
Progress 2/2500, lambda_: 4.9417133613238385e-15,  gamma: 1e-06, f1: 0.44
Progress 3/2500, lambda_: 1.0985411419875573e-14,  gamma: 1e-06, f1: 0.44
Progress 4/2500, lambda_: 2.4420530945486547e-14,  gamma: 1e-06, f1: 0.44
Progress 5/2500, lambda_: 5.4286754393238596e-14,  gamma: 1e-06, f1: 0.44
Progress 6/2500, lambda_: 1.2067926406393265e-13,  gamma: 1e-06, f1: 0.44
Progress 7/2500, lambda_: 2.682695795279727e-13,  gamma: 1e-06, f1: 0.44
Progress 8/2500, lambda_: 5.963623316594636e-13,  gamma: 1e-06, f1: 0.44
Progress 9/2500, lambda_: 1.3257113655901109e-12,  gamma: 1e-06, f1: 0.44
Progress 10/2500, lambda_: 2.9470517025518096e-12,  gamma: 1e-06, f1: 0.44
Progress 11/2500, lambda_: 6.551285568595496e-12,  gamma: 1e-06, f1: 0.44
Progress 12/2500, lambda_: 1.4563484775012445e-11,  gamma: 1e-06, f1: 0.44
Progress 13/2500, lambda_: 3.237457542817653e-11,  gamma

# Learning algorithms

### Least squares

In [21]:
y_train

array([ 1., -1., -1., ...,  1., -1., -1.])

In [ ]:
weight, loss = least_squares(y_train, x_train_cleaned)
y_pred = get_classification_pred(x_train_cleaned, weight)
print(f1_score(y_train, y_pred))

### Least squares with ridges regression

In [ ]:
weight, loss = ridge_regression(y_train, x_train_cleaned, 10)
print(loss)

### Least squares with gradient descent

In [ ]:
weight, loss = mean_squared_error_gd(y_train, x_train_cleaned, np.ones((31,)), 100, 1e-3)
loss

### Least squares with stochastic gradient descent

In [ ]:
weight, loss = mean_squared_error_sgd(y_train, x_train_cleaned, np.ones((31,)), 100, 1e-3)
loss

### Logistic regression

In [ ]:
w, l = logistic_regression(y_train, x_train_cleaned, np.ones((31,)), 1000, 1e-3)

In [ ]:
w, l = reg_logistic_regression(y_train, x_train_cleaned, 0.2,  np.ones((31,)), 1000, 1e-3)